# Örnek Otel ve Uçuş Rezervasyon Acentesi

Bu çözüm, uçak biletleri ve otel rezervasyonu yapmanıza yardımcı olacaktır. Senaryo, 20 Şubat 2024'te Londra Heathrow LHR'den New York JFK'ye ekonomi sınıfında yalnızca British Airways ile uçuş ve 27 Şubat 2025'te dönüş şeklindedir. New York'ta bir Hilton otelinde konaklamak istiyorum, lütfen uçuş ve otel için maliyetleri sağlayın.


# Azure AI Agent Hizmetini Başlatın ve **.env** Dosyasından Yapılandırma Bilgilerini Alın

### **.env**

Bir .env dosyası oluşturun.

**.env**, Azure AI Agent Hizmetinin bağlantı dizesini, AOAI tarafından kullanılan modeli ve ilgili Google API Arama hizmeti API'sini, ENDPOINT vb. içerir.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Azure AI Agent Hizmeti Model Dağıtım Adınız"

[**NOTE**] Dakikada 100.000 Jeton (Rate Limit) ve dakikada 600 İstek (Rate Limit) sınırına sahip bir modele ihtiyacınız olacak.

  Modeli Azure AI Foundry - Model ve Endpoint bölümünden edinebilirsiniz.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Azure AI Agent Hizmeti Proje Bağlantı Dizeniz"

  Proje bağlantı dizesini, AI Foundry Portal Ekranındaki proje genel bakış bölümünde bulabilirsiniz.

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI Arama API Anahtarınız"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI Arama Endpoint'iniz"

Azure AI Agent Hizmetinin Model Dağıtım Adını ve Proje Bağlantı Dizesini almak için Azure AI Agent Hizmeti oluşturmanız gerekir. [Bu şablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) kullanarak doğrudan oluşturmanız önerilir. （***Not:*** Azure AI Agent Hizmeti şu anda sınırlı bir bölgede ayarlanmıştır. Bölgeyi ayarlamak için [bu bağlantıya](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) başvurmanız önerilir.)

Agent'in SERPAPI'ye erişmesi gerekir. [Bu bağlantıyı](https://serpapi.com/searches) kullanarak kaydolmanız önerilir. Kayıt işlemi tamamlandıktan sonra benzersiz bir API ANAHTARI ve ENDPOINT alabilirsiniz.


# Kurulum

Bu not defterini çalıştırmak için, `pip install -r requirements.txt` komutunu çalıştırarak gerekli kütüphaneleri yüklediğinizden emin olmanız gerekir.


In [ ]:
from semantic_kernel import __version__

__version__

Semantik Çekirdek sürümünüzün en az 1.27.2 olması gerekir.


Lütfen .env dosyası ayarlarını ve kaynaklarını yükleyin, anahtarlarınızı ve ayarlarınızı eklediğinizden ve yerel bir .env dosyası oluşturduğunuzdan emin olun.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure'a Giriş Yapın

Şimdi Azure'a giriş yapmanız gerekiyor. Bir terminal açın ve aşağıdaki komutu çalıştırın:

```bash
az login
```

Bu komut, Azure kimlik bilgilerinizi girmenizi isteyecek ve Azure AI Agent hizmetinin doğru şekilde çalışmasını sağlayacaktır.


# Açıklama:
Bu, bir SERP (Arama Motoru Sonuç Sayfası) API hizmetine erişmek için kullanılan API anahtarını saklayan bir değişkendir. API anahtarı, hesabınızla ilişkilendirilmiş istekleri doğrulamak için kullanılan benzersiz bir tanımlayıcıdır.

Amaç: Bu satırın amacı, API anahtarını bir değişkende saklamaktır, böylece SERP API hizmetine yapılan istekleri doğrulamak için kullanılabilir. API anahtarı, hizmete erişmek ve arama yapmak için gereklidir.
SERP API Anahtarı Nasıl Alınır: Bir SERP API anahtarı almak için, https://serpapi.com adresindeki genel adımları izleyin (kullandığınız belirli SERP API hizmetine bağlı olarak adımlar değişebilir):

Bir SERP API Hizmeti Seçin: SerpAPI, Google Custom Search JSON API ve diğerleri gibi çeşitli SERP API hizmetleri mevcuttur. İhtiyaçlarınıza en uygun olanı seçin.

Bir Hesap Oluşturun: Seçtiğiniz SERP API hizmetinin web sitesine gidin ve bir hesap oluşturun. Bazı temel bilgileri sağlamanız ve e-posta adresinizi doğrulamanız gerekebilir.

Bir API Anahtarı Oluşturun: Kaydolduktan sonra, hesabınıza giriş yapın ve API bölümüne veya kontrol paneline gidin. Yeni bir API anahtarı oluşturma veya üretme seçeneğini arayın.
API Anahtarını .env dosyanıza kopyalayın.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Açıklama:
BASE_URL: Bu, SERP API uç noktası için temel URL'yi saklayan bir değişkendir. BASE_URL değişken adı, bu URL'nin API istekleri yapmak için başlangıç noktası olduğunu belirtmek için kullanılan bir konvansiyondur.  
'https://serpapi.com/search':  

Bu, BASE_URL değişkenine atanan gerçek URL dizesidir. SERP API kullanarak arama sorguları gerçekleştirmek için kullanılan uç noktayı temsil eder.

# Amaç:
Bu satırın amacı, SERP API için temel URL'yi tutan bir sabit tanımlamaktır. Bu URL, arama işlemleri gerçekleştirmek için API isteklerini oluştururken başlangıç noktası olarak kullanılacaktır.

# Kullanım:
Temel URL'yi bir değişkende tanımlayarak, SERP API'ye istek yapmanız gerektiğinde kodunuzda kolayca yeniden kullanabilirsiniz. Bu, kodunuzu daha sürdürülebilir hale getirir ve URL'yi birden fazla yerde sabit kodlamaktan kaynaklanabilecek hataları azaltır. Mevcut örnek, Bing arama API'sini kullanan https://serpapi.com/search?engine=bing'dir. Farklı bir API, https://Serpapi.com adresinden seçilebilir.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Açıklama:

Bu, eklenti kodunuzun bulunduğu yerdir.

Sınıf Tanımı: `class BookingPlugin`: Otel ve uçak rezervasyonu için yöntemler içeren BookingPlugin adlı bir sınıfı tanımlar.

Otel Rezervasyon Yöntemi:

- `@kernel_function(description="booking hotel")`: Otel rezervasyonu için bir çekirdek fonksiyon olarak işlevi tanımlayan bir dekoratör.
- `def booking_hotel(self, query: Annotated[str, "Şehir adı"], check_in_date: Annotated[str, "Otele giriş zamanı"], check_out_date: Annotated[str, "Otelden çıkış zamanı"]) -> Annotated[str, "Otel rezervasyon bilgisi sonucunu döndürür"]:`: Annotasyonlu parametreler ve dönüş tipi ile otel rezervasyonu için bir yöntem tanımlar.

Yöntem, otel rezervasyon isteği için bir parametre sözlüğü oluşturur ve SERP API'ye bir GET isteği gönderir. Yanıt durumunu kontrol eder ve başarılıysa otel özelliklerini döndürür, başarısız olursa None döndürür.

Uçak Rezervasyon Yöntemi:

- `@kernel_function(description="booking flight")`: Uçak rezervasyonu için bir çekirdek fonksiyon olarak işlevi tanımlayan bir dekoratör.
- `def booking_flight(self, origin: Annotated[str, "Kalkış yeri adı"], destination: Annotated[str, "Varış yeri adı"], outbound_date: Annotated[str, "Gidiş tarihi"], return_date: Annotated[str, "Dönüş tarihi"]) -> Annotated[str, "Uçak rezervasyon bilgisi sonucunu döndürür"]:`: Annotasyonlu parametreler ve dönüş tipi ile uçak rezervasyonu için bir yöntem tanımlar.

Yöntem, gidiş ve dönüş uçuş istekleri için parametre sözlükleri oluşturur ve SERP API'ye GET istekleri gönderir. Yanıt durumunu kontrol eder ve başarılıysa uçuş bilgilerini sonuç dizisine ekler, başarısız olursa bir hata mesajı yazdırır. Yöntem, uçuş bilgilerini içeren sonuç dizisini döndürür.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Açıklama:
İçe Aktarma Bildirimleri: Azure kimlik bilgileri, AI ajanı, sohbet mesaj içeriği, yazar rolü ve çekirdek işlev dekoratörü için gerekli modülleri içe aktarın.

Asenkron Bağlam Yöneticisi: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Bu, Azure kimlik bilgilerini yönetmek ve bir AI ajanı istemcisi oluşturmak için bir asenkron bağlam yöneticisi kurar.

Ajan Adı ve Talimatlar:
- `AGENT_NAME = "BookingAgent"`: Ajanın adını tanımlar.
- `AGENT_INSTRUCTIONS = """..."""`: Ajanın rezervasyon taleplerini nasıl ele alacağına dair ayrıntılı talimatlar sağlar.

Ajan Tanımı Oluşturma: `agent_definition = await client.agents.create_agent(...)`: Belirtilen model, ad ve talimatlarla bir ajan tanımı oluşturur.

AzureAI Ajanı Oluşturma: `agent = AzureAIAgent(...)`: İstemciyi, ajan tanımını ve tanımlı eklentiyi kullanarak bir AzureAI ajanı oluşturur.

İş Parçacığı Oluşturma: `thread: AzureAIAgentThread | None = None`: Ajan için bir iş parçacığı oluşturur. İlk olarak bir iş parçacığı oluşturmak gerekli değildir - eğer `None` değeri sağlanırsa, ilk çağrıda yeni bir iş parçacığı oluşturulur ve yanıtın bir parçası olarak döndürülür.

Kullanıcı Girdileri: `user_inputs = ["..."]`: Ajanın işlemesi için bir kullanıcı girdileri listesi tanımlar.

Son olarak, kaynakları temizlemek için iş parçacığını ve ajanı silin.


# Kimlik Doğrulama

`DefaultAzureCredential` sınıfı, Azure SDK for Python'ın bir parçasıdır. Azure hizmetleriyle kimlik doğrulaması yapmak için varsayılan bir yöntem sunar. Çevre değişkenleri, yönetilen kimlik ve Azure CLI kimlik bilgileri gibi belirli bir sırayla birden fazla yöntemle kimlik doğrulamayı dener.

Asenkron İşlemler: aio modülü, DefaultAzureCredential sınıfının asenkron işlemleri desteklediğini belirtir. Bu, kimlik doğrulama isteklerini engellemeden gerçekleştirmek için asyncio ile kullanılabileceği anlamına gelir.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
